#### Beschreibung:
Aufbereitung der Daten.

#### Workflow:
Echtdaten des Projektes, das zurzeit in der Konfiguration gesetzt ist, werden aus der MongoDB (Datenbank) ausgelesen. Anschließend werden sie in diesem Notebook für die weitere Verwendung aufbereitet.

# Daten bereinigen

In [1]:
%run ../Setup.ipynb

import pandas as pd
from isac.database.connection import database_connector as connector
from isac import configuration
import numpy as np

### Extraktion

#### Dokumente einlesen

In [2]:
connection = connector.DatabaseConnector(name = configuration.project_database).connect()
collection = connection.getCollection(configuration.collections.force_import)

existingColumns = ['time', 'fx', 'fy', 'fz'];

df = pd.DataFrame(list(collection.find()))
df = df.sort_values(by=['time'])
df = df.reset_index(drop=True)
order = list(existingColumns)
# df[order].head()

### Cleaning

#### ? Summe der Kräfte berechnen

fz hat keine relevanten Auswirkungen auf die Kräfte, führt sogar eher zu Störeffekten, deshalb
wird sie gar nicht in die Berechnung miteinbezogen

In [3]:
df['fsum'] = np.sqrt(pow(df[order]['fx'], 2) + pow(df[order]['fy'], 2))

# Ausgabe
focusColumns = ['time', 'fx', 'fy', 'fz', 'fsum'];
order = list(focusColumns)
df[order].head()

,time,fx,fy,fz,fsum
0,0.000,-1.37329,-78.5522,46.5088,78.564203
1,0.002,-1.37329,-78.5522,46.6309,78.564203
2,0.004,-1.73950,-79.2847,46.1426,79.303780
3,0.006,-1.92261,-79.5593,46.3257,79.582527
4,0.008,-2.28882,-78.5522,46.2036,78.585538


#### Resampling auf 2 Millisekunden
#### Lineare Interpolation

In [4]:
df['time'] = pd.to_timedelta(df['time'], unit="s")
df = df.set_index('time')
df = df.resample('2L')
df = df.interpolate(method='linear') # spline, polynomial, -order
df = df.reset_index()